In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Data Download

In [ ]:
!gdown 1mmYhPMDT9ouz5L2bAL5w6A0S6GoY8IOo

Downloading...
From: https://drive.google.com/uc?id=1mmYhPMDT9ouz5L2bAL5w6A0S6GoY8IOo
To: /content/kaggle.json
100% 62.0/62.0 [00:00<00:00, 249kB/s]


In [ ]:
!pip install -q kaggle
# The kaggle API client expects the file to be in ~/.kaggle, so move it there

!mkdir -p ~/.kaggle #Create the directory
!cp kaggle.json ~/.kaggle/

In [ ]:
# Now all you need is to set permissions to the json file
!chmod 600 /root/.kaggle/kaggle.json

# check your directory before downloading the datasets
!pwd #Print Working Directory

/content


In [ ]:
!kaggle datasets download -d pavansanagapati/images-dataset

 99% 1.00G/1.01G [00:11<00:00, 92.3MB/s]
100% 1.01G/1.01G [00:11<00:00, 95.9MB/s]


In [ ]:
!unzip images-dataset.zip

##Ambil label dan gambar random

In [ ]:
import os
import random
from skimage import io
import IPython.display as display
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt


def random_print_Image():
  # Define the path to the main folder containing subfolders with labeled images
  main_folder_path = '/content/data/data'

  # Get a list of subfolder names (labels)
  labels = os.listdir(main_folder_path)

  # Select a random label (subfolder) from the list, kunci jawaban
  random_label = random.choice(labels)

  # Get a list of image filenames within the subfolder
  subfolder_path = os.path.join(main_folder_path, random_label)
  image_files = os.listdir(subfolder_path)

  # Select a random image filename from the list
  random_image_filename = random.choice(image_files)

  # Get the path to the randomly selected image file
  image_path = os.path.join(subfolder_path, random_image_filename)

  # Load the image from a URL
  image = Image.open(image_path)

  # Display the image
  # plt.imshow(image)
  # plt.show()

  image.show()

  return random_label

## Answer Checking, STT

In [ ]:
def predict(AUDIO_PATH):
  import torch, torchaudio

  waveform, sr = torchaudio.load(AUDIO_PATH)

  bundle = torchaudio.pipelines.WAV2VEC2_ASR_BASE_960H

  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model = bundle.get_model().to(device)
  waveform = waveform.to(device)

  if sr != bundle.sample_rate:
    waveform = torchaudio.functional.resample(waveform, sr, bundle.sample_rate)

  with torch.inference_mode():
    emission, _ = model(waveform)

  class GreedyCTCDecoder(torch.nn.Module):
    def __init__(self, label, blank=0):
      super().__init__()
      self.label = label
      self.blank = blank

    def forward(self, emission=torch.Tensor):
      index = torch.argmax(emission, dim=-1)
      index = torch.unique_consecutive(index, dim=-1)

      index = [i for i in index
               if i != self.blank]

      return ''.join([self.label[i]
                      for i in index])

  decoder = GreedyCTCDecoder(label=bundle.get_labels())
  transcribe = decoder(emission[0])

  return transcribe.replace('|', '')

#GAME

In [ ]:
!pip install import_ipynb
import import_ipynb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.7 MB/s eta 0:00:00


In [ ]:
#donlot colab yg recording as ipynb, trus upload ke sini
!gdown 1yATu_T6LKdFBWZNGuiihuBBqxrKbqjNC
!unzip
import Recording1 #import file recording1.ipynb

importing Jupyter notebook from Recording1.ipynb
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from IPython.display import clear_output
import time
import sys


def menu():
  clear_output()

  score = 0
  high_score = 0

  print('Image Guessing Game\n\n1. Start Game\n2. View High Score\n3. Exit')
  x = int(input('Enter option: '))

  clear_output()

  # def gameFunc():
    # answer = random_print_Image()
    # print('Talk Here: https://colab.research.google.com/drive/1bTv5kbWb5IOcGXSrMBDAiFLR40cx_Rqy#scrollTo=FP86TbtgqbYY&line=2&uniqifier=1')

    # time.sleep(20)

    # drive.mount('/content/drive/',force_remount=True) #refresh audio baru
    # attempted = predict('/content/drive/MyDrive/records.mp3').lower()
    # if answer in attempted.replace(' ', ''):
    #   score += 100
    #   print('Nice One!\n')

    # else:
    #   print('Nice Try!\n')
    #   print('Attempt', attempted, ' Answer', answer)

    # y = input('Press enter to continue...')

  if x == 1:
    # start_time = time.time()
    # end_time = start_time + 60 #main semenit

    while True:
      answer = random_print_Image()
      Recording1.record()

      time.sleep(1)

      # drive.flush_and_unmount()

      attempted = predict('/content/drive/MyDrive/records.mp3').lower()
      if answer in attempted.replace(' ', ''):
        score += 100
        print(f'Nice One! the answer was {answer}\n')

      else:
        print('Nice Try!\n')
        print('Attempt', attempted, ' Answer', answer)

      print(f'Your Score is {score}\n')
      if score > high_score:
        high_score = score

      y = input('Press 1 to main menu or enter to continue...')
      if y == '1':
        menu()

    # print('Game Over!')
    # y = input('Press enter to continue...')



  elif x == 2:
    print('Your High Score is: ', high_score)
    y = input('\nPress enter to continue...')

  elif x == 3:
    sys.exit('Thanks for playing!')

  else:
    x = int(input('Enter option: '))


  menu()

menu()
